In [1]:
import requests
import os
import pandas as pd
import json
import csv
import sqlite3
from datetime import datetime 

In [2]:
#1 Extracting the data from github API
username ="Aadi6182831"
API = os.environ.get('GITHUB_API_KEY')
url=f"https://api.github.com/users/{username}/repos"
header={"Authentication":f"token {API}"}

response= requests.get(url,headers=header)
print(requests.get(url,headers=header)) #it gives only status_codes , nothing it results
print(response) #prints the same which we assigned of HTTP method status code
Data=response.json() #data which is resulted from HTTP method is in json and that json is converting into python object directly(list or dictionary)
data=json.dumps(Data[:3],indent=4)#dictionary or list data is converting into json and viweing only 3 rows .This json data can be used for API POST requests for loading data.
print(data)

<Response [200]>
<Response [200]>
[
    {
        "id": 904493648,
        "node_id": "R_kgDONel6UA",
        "name": "pandas_exercises",
        "full_name": "Aadi6182831/pandas_exercises",
        "private": false,
        "owner": {
            "login": "Aadi6182831",
            "id": 189071221,
            "node_id": "U_kgDOC0T_dQ",
            "avatar_url": "https://avatars.githubusercontent.com/u/189071221?v=4",
            "gravatar_id": "",
            "url": "https://api.github.com/users/Aadi6182831",
            "html_url": "https://github.com/Aadi6182831",
            "followers_url": "https://api.github.com/users/Aadi6182831/followers",
            "following_url": "https://api.github.com/users/Aadi6182831/following{/other_user}",
            "gists_url": "https://api.github.com/users/Aadi6182831/gists{/gist_id}",
            "starred_url": "https://api.github.com/users/Aadi6182831/starred{/owner}{/repo}",
            "subscriptions_url": "https://api.github.com/users/Aadi

In [3]:
doit=json.loads(data) # json string format is converting into python object.
print(doit)

[{'id': 904493648, 'node_id': 'R_kgDONel6UA', 'name': 'pandas_exercises', 'full_name': 'Aadi6182831/pandas_exercises', 'private': False, 'owner': {'login': 'Aadi6182831', 'id': 189071221, 'node_id': 'U_kgDOC0T_dQ', 'avatar_url': 'https://avatars.githubusercontent.com/u/189071221?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/Aadi6182831', 'html_url': 'https://github.com/Aadi6182831', 'followers_url': 'https://api.github.com/users/Aadi6182831/followers', 'following_url': 'https://api.github.com/users/Aadi6182831/following{/other_user}', 'gists_url': 'https://api.github.com/users/Aadi6182831/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/Aadi6182831/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/Aadi6182831/subscriptions', 'organizations_url': 'https://api.github.com/users/Aadi6182831/orgs', 'repos_url': 'https://api.github.com/users/Aadi6182831/repos', 'events_url': 'https://api.github.com/users/Aadi6182831/events{/privacy}', '

In [4]:
#GitHub API response contains a lot of information, but not all of it may be useful for your analysis or project.
# we want to filter out unnecessary information and focus only on the relevant data fields (e.g., stars, forks, language, etc.).
repo_data =[] 
#By converting this data into a structured list of dictionaries, we can easily convert it into a DataFrame 
#for further analysis and loading into a database.
for repo in Data:
    repo_data.append({'id':repo['id'],'name':repo['name'],'fullname':repo['full_name'],'account_private':repo["private"],
                      "owner":repo['owner']['id'],"Description":repo["description"],"star_count":repo["stargazers_count"],"lang":repo["language"],
                      "forkscount":repo['forks_count'],'forks':repo["forks"],'issues':repo["open_issues"],'time_creation':repo['created_at']})
print(repo_data)

[{'id': 904493648, 'name': 'pandas_exercises', 'fullname': 'Aadi6182831/pandas_exercises', 'account_private': False, 'owner': 189071221, 'Description': 'Practice your pandas skills!', 'star_count': 0, 'lang': None, 'forkscount': 0, 'forks': 0, 'issues': 0, 'time_creation': '2024-12-17T02:11:38Z'}, {'id': 905032955, 'name': 'python_essentials_for_data_engineers', 'fullname': 'Aadi6182831/python_essentials_for_data_engineers', 'account_private': False, 'owner': 189071221, 'Description': 'Code for blog at https://www.startdataengineering.com/post/python-for-de/', 'star_count': 0, 'lang': 'Python', 'forkscount': 0, 'forks': 0, 'issues': 0, 'time_creation': '2024-12-18T02:59:48Z'}]


In [5]:
# DataFrames are great for manipulating, filtering, and analyzing data in Python.
#They provide high-level functions like groupby(), filter(), sort(), and pivot() that make complex data analysis 
# and transformation easier and more efficient.
df=pd.DataFrame(repo_data)
df.head()

,id,name,fullname,account_private,owner,Description,star_count,lang,forkscount,forks,issues,time_creation
0,904493648,pandas_exercises,Aadi6182831/pandas_exercises,False,189071221,Practice your pandas skills!,0,None,0,0,0,2024-12-17T02:11:38Z
1,905032955,python_essentials_for_data_engineers,Aadi6182831/python_essentials_for_data_engineers,False,189071221,Code for blog at https://www.startdataengineer...,0,Python,0,0,0,2024-12-18T02:59:48Z


In [6]:
df.to_csv("repos_data.csv",index=False)

In [7]:
#conn.open()
# Connect to SQLite
conn = sqlite3.connect('github_repos.db')  # Connect to SQLite database (or create it if it doesn't exist)
cursor = conn.cursor()  # Create a cursor to interact with the database

# Create Table
cursor.execute('''CREATE TABLE IF NOT EXISTS REPOS( id INTEGER PRIMARY KEY,
        name TEXT,
        full_name TEXT,
        account_private	TEXT,
        owner TEXT,
        Description	TEXT,
        star_count INTEGER,
        lang TEXT,
        forkscount INTEGER,
        forks INTEGER,
        issues INTEGER,
        time_creation TEXT
    )
''')


# Commit
conn.commit()

In [8]:
df.to_sql('REPOS', conn, if_exists='replace', index=False)
print("Data successfully stored in SQLite!")

Data successfully stored in SQLite!


In [9]:
# SQL queries to retrieve the data.
# Retrieve repositories with =0
cursor.execute("SELECT * FROM REPOS")
high_star_repos = cursor.fetchall()
print(high_star_repos)
conn.close()

[(904493648, 'pandas_exercises', 'Aadi6182831/pandas_exercises', 0, 189071221, 'Practice your pandas skills!', 0, None, 0, 0, 0, '2024-12-17T02:11:38Z'), (905032955, 'python_essentials_for_data_engineers', 'Aadi6182831/python_essentials_for_data_engineers', 0, 189071221, 'Code for blog at https://www.startdataengineering.com/post/python-for-de/', 0, 'Python', 0, 0, 0, '2024-12-18T02:59:48Z')]


In [10]:
#data loading from csv document(can load by using 2 processes,1.csv loading to dataframe
#2 By creating database,table in sqlite and inserting the data later
conn1 = sqlite3.connect('github_csv_repos.db')  # Connect to SQLite database (or create it if it doesn't exist)
cursor1 = conn1.cursor()  # Create a cursor to interact with the database

cursor1.execute("DELETE FROM CSV_REPOS")
conn1.commit()

# Create Table
cursor1.execute('''CREATE TABLE IF NOT EXISTS CSV_REPOS( id INTEGER PRIMARY KEY,
        name TEXT,
        full_name TEXT,
        account_private	TEXT,
        owner TEXT,
        Description	TEXT,
        star_count INTEGER,
        lang TEXT,
        forkscount INTEGER,
        forks INTEGER,
        issues INTEGER,
        time_creation TEXT
    )
''')

conn1.commit()

In [11]:
with open('repos_data.csv',"r") as file:
    reader=csv.reader(file)
# used to skip the first row (header row) in a CSV file when reading it using Python's csv module.
    next(reader)
    for row in reader :
        cursor1.execute('''INSERT INTO CSV_REPOS VALUES(?,?,?,?,?,?,?,?,?,?,?,?)''', row) 

    conn1.commit()

In [12]:
cursor1.execute("SELECT * FROM CSV_REPOS")
cursor1.fetchall()

[(904493648,
  'pandas_exercises',
  'Aadi6182831/pandas_exercises',
  'False',
  '189071221',
  'Practice your pandas skills!',
  0,
  '',
  0,
  0,
  0,
  '2024-12-17T02:11:38Z'),
 (905032955,
  'python_essentials_for_data_engineers',
  'Aadi6182831/python_essentials_for_data_engineers',
  'False',
  '189071221',
  'Code for blog at https://www.startdataengineering.com/post/python-for-de/',
  0,
  'Python',
  0,
  0,
  0,
  '2024-12-18T02:59:48Z')]

In [13]:
#Perform Data Analysis
query = "SELECT lang, COUNT(*) as repo_count FROM CSV_REPOS GROUP BY lang ORDER BY repo_count DESC"
df_lang = pd.read_sql(query, conn1)
print(df_lang)

     lang  repo_count
0  Python           1
1                   1


In [14]:
conn1.close()
# When you call ***sqlite3.connect('github_csv_repos.db')***, it opens the database file named github_csv_repos.db. 
# If the file does not exist, SQLite will create it.

In [15]:
# Example to log the script's execution time in a log file
with open("script_log.txt", "a") as log:
    log.write(f"Task executed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    print(f"Task executed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

Task executed at 2025-02-01 13:39:12

